In [1]:
# import
import pickle
import pandas as pd

In [2]:
# data diectory, modify before use
data_dir = "scripts/1_preprocessing"

In [3]:
# for each horizon
for horizon in [1, 6, 12]:
    # load patient ID
    separate_id = pickle.load(open(
        data_dir + "/processed_data/ids/{}h_ID.pickle".format(horizon), "rb"
    ))
    sepsis_id, nonsep_id = separate_id['sepsis'], separate_id['nonsep']
    # combine id
    all_id = sepsis_id + nonsep_id
    # time stamps
    time_stamp = pd.DataFrame({'offset': range(-horizon * 60, 5, 5)})
    # for each patient, same step
    for p_id in all_id:
        # load data
        patient_data = pd.read_csv(
            data_dir + "processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id),
            index_col=False
        )
        # merge with time stamps
        patient_data = pd.merge(
            patient_data, time_stamp, how="outer", on=['offset']
        ).drop_duplicates()
        # sort according to offset
        patient_data = patient_data.sort_values(by="offset", axis=0)
        # forward backward fill
        patient_data = patient_data.fillna(method="ffill")
        patient_data = patient_data.fillna(method="bfill")
        # take what's needed
        patient_data = patient_data.loc[patient_data['offset'].isin(time_stamp['offset'].values)]
        # fill na with 0
        patient_data = patient_data.fillna(value=0)
        # out
        patient_data.to_csv('processed_data/2_{}_h/{}.csv'.format(horizon, p_id), index=False)
